In [4]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, Dataset, DataLoader
from torch.optim import SGD, Adam

device = 'mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
print(f'Running on: {device}')

from torchvision import datasets
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Running on: cpu


In [5]:
# train data in form N x C x H x W (2x1x4x4 in example)
x_train = torch.tensor([ [[[1,2,3,4],
                           [2,3,4,5],
                           [5,6,7,8],
                           [1,3,4,5]]],
                         [[[-1,2,3,-4],
                           [2,-3,4,5],
                           [-5,6,-7,8],
                           [-1,-3,-4,-5]]] 
                       ]).to(device).float()
x_train /= 8.
y_train = torch.tensor([0,1]).to(device).float()

In [17]:
def get_model():
    model = nn.Sequential(
        #convolution layer - channels in: 1, channels out: 1, ksize: 3 ==> in tensor: 4x4 out tensor: 2x2
        nn.Conv2d(1, 1, kernel_size=3), 
        #pooling 2x2 on tensor 2x2 ==> out: 1x1
        nn.MaxPool2d(2),
        nn.ReLU(),
        nn.Flatten(), # vector 1x1
        nn.Linear(1, 1),
        nn.Sigmoid(),
    ).to(device)
    loss_fn = nn.BCELoss()
    optimizer = Adam(model.parameters(), lr=1e-3)
    return model, loss_fn, optimizer

In [18]:
from torchsummary import summary
model, loss_fn, optimizer = get_model()
summary(model, x_train);

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 1, 2, 2]             10
├─MaxPool2d: 1-2                         [-1, 1, 1, 1]             --
├─ReLU: 1-3                              [-1, 1, 1, 1]             --
├─Flatten: 1-4                           [-1, 1]                   --
├─Linear: 1-5                            [-1, 1]                   2
├─Sigmoid: 1-6                           [-1, 1]                   --
Total params: 12
Trainable params: 12
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00


In [19]:
def train_batch(x, y, model, opt, loss_fn):
    model.train()
    prediction = model(x)
    batch_loss = loss_fn(prediction.squeeze(0), y)
    batch_loss.backward()
    opt.step()
    opt.zero_grad()
    return batch_loss.item()

In [22]:
trn_dl = DataLoader(TensorDataset(x_train, y_train))

for epoch in range(2000):
    for ix, batch in enumerate(iter(trn_dl)):
        x, y = batch
        batch_loss = train_batch(x, y, model, optimizer, loss_fn)

print(batch_loss)

3.49289330188185e-05


In [24]:
model(x_train[:1])

tensor([[0.0100]], grad_fn=<SigmoidBackward0>)

In [19]:
print(
    x_train.shape,
    x_train.shape[-2:],
    x_train.shape[:-1],
    x_train, 
    x_train[0,0])

kk = x_train[0,0]
kk.view(1,1,4,4)

torch.Size([2, 1, 4, 4]) torch.Size([4, 4]) torch.Size([2, 1, 4]) tensor([[[[ 0.1250,  0.2500,  0.3750,  0.5000],
          [ 0.2500,  0.3750,  0.5000,  0.6250],
          [ 0.6250,  0.7500,  0.8750,  1.0000],
          [ 0.1250,  0.3750,  0.5000,  0.6250]]],


        [[[-0.1250,  0.2500,  0.3750, -0.5000],
          [ 0.2500, -0.3750,  0.5000,  0.6250],
          [-0.6250,  0.7500, -0.8750,  1.0000],
          [-0.1250, -0.3750, -0.5000, -0.6250]]]]) tensor([[0.1250, 0.2500, 0.3750, 0.5000],
        [0.2500, 0.3750, 0.5000, 0.6250],
        [0.6250, 0.7500, 0.8750, 1.0000],
        [0.1250, 0.3750, 0.5000, 0.6250]])


tensor([[[[0.1250, 0.2500, 0.3750, 0.5000],
          [0.2500, 0.3750, 0.5000, 0.6250],
          [0.6250, 0.7500, 0.8750, 1.0000],
          [0.1250, 0.3750, 0.5000, 0.6250]]]])